In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.perception.segmentation import UnetSegmentation
from lac.slam.semantic_feature_tracker import SemanticFeatureTracker
from lac.slam.frontend import Frontend
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.utils.visualization import image_grid
from lac.util import load_data, load_stereo_images, load_images, positions_rmse_from_poses

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
# data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset1_recovery_agent"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
# images = load_images(data_path, cameras=["FrontLeft", "FrontRight"], start_frame=0, end_frame=10000)
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0, end_frame=10000)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

In [ ]:
segmentation = UnetSegmentation()
tracker = SemanticFeatureTracker(cam_config)

In [ ]:
# Initialize modules
START_FRAME = 250
END_FRAME = 2000

pred = segmentation.predict(left_imgs[START_FRAME])
tracker.initialize(left_imgs[START_FRAME], right_imgs[START_FRAME], pred)

In [ ]:
curr_pose = poses[START_FRAME]
trajectory = [curr_pose]
eval_poses = [curr_pose]

# Main loop over image frames
for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    pred = segmentation.predict(left_imgs[frame])
    odom = tracker.track_pnp(left_imgs[frame], right_imgs[frame], pred)
    curr_pose = curr_pose @ odom
    trajectory.append(curr_pose)
    eval_poses.append(poses[frame])

In [ ]:
fig = plot_poses(eval_poses, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(trajectory, fig=fig, no_axes=True, color="orange", name="VO poses")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
positions_rmse_from_poses(eval_poses, trajectory)